# Average Annual Daily Traffic (Current)

Minnesota Department of Transport (MnDoT)

Output Coordinate Reference System: UTM 15N - EPSG:26915

Source: https://gisdata.mn.gov/dataset/trans-aadt-traffic-segments

Metadata: https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_aadt_traffic_segments/metadata/metadata.html

In [1]:
### Import Libraries

# File manipulation

import os # For working with Operating System
import urllib # For accessing websites
import zipfile # For extracting from Zipfiles
from io import BytesIO # For reading bytes objects

import psycopg2
from psycopg2 import sql, extras

# Analysis

import numpy as np # For working with Arrays
import pandas as pd # Data Manipulation
import geopandas as gpd

# Get CWD

cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

In [2]:
# Definitions

def Get_nearby_sensors(target_geom, sensors_df, distance):
    ''' This function will return all sensor indices within the distance (meters) of the target
    '''

    # Get geometries of targets and sources 

    sensors_geo = sensors_df.geometry

    target_buffer = target_geom.buffer(distance)

    is_within = sensors_geo.within(target_buffer)

    return sensors_df[is_within].sensor_index

In [13]:
# Load Traffic segments

datapath = os.path.join('..', '..', 'Data')

intersections_df = gpd.read_file(os.path.join(datapath, 'Road_Intersections.geojson'))

sensors_df = gpd.read_file(os.path.join(datapath, 'PurpleAir_Stations.geojson'))

In [36]:
# Iterate through to find nearby sensors

nearby_df = intersections_df.copy()# .drop(intersections_df.columns, axis = 1)

nearby_df['nearby_sensors'] = None

for i, row in nearby_df.iterrows():

    target_geom = row.geometry

    sensor_ids = Get_nearby_sensors(target_geom, sensors_df, 2000)
    
    nearby_df.loc[i, 'nearby_sensors'] = str(sensor_ids.tolist())
    

In [37]:
# Format for database

# Convert strings of lists into lists

nearby_df['nearby_sensors'] = nearby_df['nearby_sensors'].apply(lambda x: x[2:-2].split(', '))

# Drop extra columns

nearby_df[

nearby_df.head()

,NS_cross_street,EW_cross_street,geometry,nearby_sensors
0,Nicollet Ave S,W 61st St,POINT (478042.278 4971026.506),"[142734', '143246', '166459]"
1,Lyndale Ave S,W 61st St,POINT (477234.219 4971031.999),"[142734', '143246', '166459]"
2,Portland Ave S,E 60th St,POINT (478860.281 4971245.500),"[142734', '143246', '166459]"
3,Chicago Ave S,E 60th St,POINT (479249.344 4971244.000),"[142734', '143246', '166459]"
4,Nicollet Ave S,E 60th St,POINT (478043.782 4971248.999),"[142734', '143246', '166459]"


In [25]:
# Let's Save that locally

intersections_df[~is_duplicated].to_file(os.path.join(datapath, 'Road_Intersections.geojson'))

# Upload to DB

In [26]:
intersections_df_no_duplicates = intersections_df[~is_duplicated]

intersections_df_no_duplicates.head(1)

,NS_cross_street,EW_cross_street,inter
0,Nicollet Ave S,W 61st St,POINT (478042.278 4971026.506)


In [27]:
# Correct column types

sorted_df = intersections_df_no_duplicates.drop('inter', axis=1).copy()

sorted_df['wkt'] = intersections_df_no_duplicates.inter.apply(lambda x: x.wkt)

cols_for_db = list(intersections_df_no_duplicates.columns[:-1]) + ['geometry']

In [28]:
cols_for_db

['NS_cross_street', 'EW_cross_street', 'geometry']

In [29]:
# Upload to Database

# Get credentials

cred_pth = os.path.join(os.getcwd(), '..', '..', 'Scripts', 'database', 'db_credentials.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].rstrip('\n').split(', ')
    
# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

conn = psycopg2.connect(**pg_connection_dict)

In [30]:
# Create Cursor for commands

cur = conn.cursor()


for index, row in sorted_df.iterrows():
    
    # This is really a great way to insert a lot of data
    
    q1 = sql.SQL('INSERT INTO "Road Intersections" ({}) VALUES ({},{});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db)-1)),
     sql.SQL('ST_Transform(ST_SetSRID(ST_GeomFromText(%s), 26915),4326)::geometry'))
#     print(q1.as_string(conn))
#     break

    cur.execute(q1.as_string(conn),
        (list(row.values))
        )
    # Commit command

    conn.commit()

# Close cursor

cur.close()

# Close connection

conn.close()